**Electricity Demand Forecasting with World Bank WDI CSV (For Ghana & NIgeria)**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving worldbank_energy.csv to worldbank_energy.csv


In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv("worldbank_energy.csv", skiprows=4)

df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,Electric power consumption (kWh per capita),EG.USE.ELEC.KH.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,Electric power consumption (kWh per capita),EG.USE.ELEC.KH.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,568.703452,566.073368,568.141299,548.496602,512.766661,513.745842,497.161668,NaN,NaN,NaN
2,Afghanistan,AFG,Electric power consumption (kWh per capita),EG.USE.ELEC.KH.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Africa Western and Central,AFW,Electric power consumption (kWh per capita),EG.USE.ELEC.KH.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,215.380351,179.972422,182.920554,188.361690,193.378593,200.861531,203.999368,NaN,NaN,NaN
4,Angola,AGO,Electric power consumption (kWh per capita),EG.USE.ELEC.KH.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,331.664900,315.199297,370.736573,410.864566,437.653351,392.355835,392.507047,NaN,NaN,NaN


In [ ]:
print(df.columns.tolist())

['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', 'Unnamed: 69']


In [ ]:
countries = ["Ghana", "Nigeria"]
df = df[df["Country Name"].isin(countries)]

df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
83,Ghana,GHA,Electric power consumption (kWh per capita),EG.USE.ELEC.KH.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,357.703181,408.852815,444.388812,469.785529,503.546669,542.211681,552.502821,NaN,NaN,NaN
174,Nigeria,NGA,Electric power consumption (kWh per capita),EG.USE.ELEC.KH.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,219.234491,138.189100,139.617321,138.391347,141.081957,142.790930,135.603309,NaN,NaN,NaN


In [ ]:
df_long = df.melt(
    id_vars=["Country Name", "Country Code"],
    var_name="Year",
    value_name="Demand"
)

df_long["Year"] = pd.to_numeric(df_long["Year"], errors="coerce")
df_long = df_long.dropna(subset=["Year", "Demand"])

df_long.head()

,Country Name,Country Code,Year,Demand
26,Ghana,GHA,1971.0,301.211909
27,Nigeria,NGA,1971.0,28.611536
28,Ghana,GHA,1972.0,333.856492
29,Nigeria,NGA,1972.0,32.763528
30,Ghana,GHA,1973.0,377.575359


In [ ]:
df_long = df_long.sort_values(["Country Name", "Year"])

df_long["lag1"] = pd.to_numeric(df_long.groupby("Country Name")["Demand"].shift(1), errors="coerce")
df_long["lag2"] = pd.to_numeric(df_long.groupby("Country Name")["Demand"].shift(2), errors="coerce")
df_long["lag3"] = pd.to_numeric(df_long.groupby("Country Name")["Demand"].shift(3), errors="coerce")
df_long["lag4"] = pd.to_numeric(df_long.groupby("Country Name")["Demand"].shift(4), errors="coerce")
df_long["lag5"] = pd.to_numeric(df_long.groupby("Country Name")["Demand"].shift(5), errors="coerce")

df_long["rolling5"] = (
    df_long.groupby("Country Name")["Demand"]
    .transform(lambda x: x.shift(1).rolling(5).mean())
)

df_long["growth_rate"] = pd.to_numeric((df_long["Demand"] / df_long["lag1"]) - 1, errors="coerce")

df_long = df_long.dropna()
df_long.head()

,Country Name,Country Code,Year,Demand,lag1,lag2,lag3,lag4,lag5,rolling5,growth_rate
96,Ghana,GHA,2006.0,284.80035,235.907910,210.990357,212.333316,298.851080,325.282767,256.673086,0.207252
98,Ghana,GHA,2007.0,234.579754,284.800350,235.907910,210.990357,212.333316,298.851080,248.576603,-0.176336
100,Ghana,GHA,2008.0,253.113966,234.579754,284.800350,235.907910,210.990357,212.333316,235.722338,0.079010
102,Ghana,GHA,2009.0,254.92039,253.113966,234.579754,284.800350,235.907910,210.990357,243.878467,0.007137
104,Ghana,GHA,2010.0,272.188473,254.920390,253.113966,234.579754,284.800350,235.907910,252.664474,0.067739


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X = df_long[["Year", "lag1", "lag2", "lag3", "lag4", "lag5", "rolling5", "growth_rate"]]
y = df_long["Demand"]

countries = df_long["Country Name"]
years = df_long["Year"]

X_train, X_test, y_train, y_test, countries_train, countries_test, years_train, years_test = train_test_split(X, y, countries, years, shuffle=False, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 14.126080529191764


In [ ]:
results = pd.DataFrame({
    "Year": years_test,
    "Country": countries_test,
    "Actual Demand": y_test,
    "Predicted Demand": y_pred
})

results.head(10000000)

,Year,Country,Actual Demand,Predicted Demand
117,2016.0,Nigeria,219.234491,219.784181
119,2017.0,Nigeria,138.1891,140.164852
121,2018.0,Nigeria,139.617321,138.915767
123,2019.0,Nigeria,138.391347,140.235818
125,2020.0,Nigeria,141.081957,149.687058
127,2021.0,Nigeria,142.79093,140.300631
129,2022.0,Nigeria,135.603309,132.357821


In [ ]:
def forecast(country, target_year):
    df_c = df_long[df_long["Country Name"] == country].copy()
    last_year = int(df_c["Year"].max())

    if target_year <= last_year:
        raise ValueError(f"{country} data already goes up to {last_year}. Pick a later year.")

    future_df = df_c.copy()
    predictions = []

    for year in range(last_year + 1, target_year + 1):
        X_future = future_df[features].iloc[-1:].copy()

        pred = model.predict(X_future)[0]
        predictions.append({"Year": year, "Country": country, "Predicted Demand": pred})

        new_row = {
            "Year": year,
            "Country Name": country,
            "Demand": pred,
            "lag1": pred,
            "lag2": future_df.iloc[-1]["lag1"],
            "lag3": future_df.iloc[-1]["lag2"],
            "lag4": future_df.iloc[-1]["lag3"],
            "lag5": future_df.iloc[-1]["lag4"],
        }
        new_row["rolling5"] = np.mean([
            new_row["lag1"], new_row["lag2"], new_row["lag3"], new_row["lag4"], new_row["lag5"]
        ])
        new_row["growth_rate"] = (new_row["lag1"] / new_row["lag2"] - 1) if new_row["lag2"] != 0 else 0

        future_df = pd.concat([future_df, pd.DataFrame([new_row])], ignore_index=True)

    return pd.DataFrame(predictions)


In [ ]:
print(forecast("Ghana", 2030))
print(forecast("Nigeria", 2030))


   Year Country  Predicted Demand
0  2023   Ghana        563.199908
1  2024   Ghana        588.779311
2  2025   Ghana        618.071226
3  2026   Ghana        650.873568
4  2027   Ghana        685.179285
5  2028   Ghana        721.048756
6  2029   Ghana        758.661270
7  2030   Ghana        798.260926
   Year  Country  Predicted Demand
0  2023  Nigeria        129.029005
1  2024  Nigeria        104.492846
2  2025  Nigeria         59.472451
3  2026  Nigeria        -38.273362
4  2027  Nigeria       -401.210898
5  2028  Nigeria       1622.838243
6  2029  Nigeria        612.407202
7  2030  Nigeria        427.537809
